# In this notebook, we will be creating a sample diversified(very) portfolio based on securities' Sharpe ratios. We will look at the monthly returns of stocks over the past year. We will then make an update to this notebook in 1 month to see how the portfolio did

## The Sharpe ratio is calculated by first subtracting a risk-free investment from the mean returns(in this case, monthly), and then dividing by the standard deviation of those returns
$$
General Formula = 
S = \frac{R_p - R_f}{\sigma_p}
$$

$$
Annualizing Monthly Sharpe Ratio = 
S_{annual} = S_{monthly} \times \sqrt{12}
$$


## This project's free return will be based on the US Treasury Bill Returns
## As of September 4, 2025, the 1-Month U.S. Treasury Bill yield—expressed as an annualized rate—is approximately 4.33%.

$$
R_{f, \text{monthly}} \approx 0.3608\%
$$


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pickle
from tqdm import tqdm
import os
import pandas_datareader.data as web
import time
from datetime import datetime
import warnings
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import sys
import networkx as nx
from selenium import webdriver
from selenium.webdriver.common.by import By
from stockdex import Ticker


import requests
from bs4 import BeautifulSoup as bs
import pprint
warnings.filterwarnings("ignore")

In [2]:
# Get Sp500 tickers
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)

sp500_tickers_df = tables[0]
sp500_tickers_df["Symbol"] = sp500_tickers_df["Symbol"].str.replace(".", "-")
tickers = sp500_tickers_df.Symbol.tolist()

In [3]:
# Read in all stock data
stock_data = pd.DataFrame()

start = time.time()
print("Loading...")
# Columns returned by .yahoo_api_price: timestamp	volume	close	open	high	low	currency	timezone	exchangeTimezoneName	exchangeName	instrumentType 
for ticker in tickers:
    if ticker not in ["FOXA", "NWS", "GOOG"]: # Drop Dupliactes
        temp_tick = Ticker(ticker)
        security_df = temp_tick.yahoo_api_price(range='1y', dataGranularity='1d')
        security_df = security_df[["timestamp", "low", "high", "open", "close", "instrumentType"]]
        security_df["Symbol"] = ticker
    
        stock_data = pd.concat([stock_data, security_df])
end = time.time()

print(f"Finished: {end - start}")

Loading...
Finished: 101.39542508125305


In [4]:
# Copy df to not have to rerun above loop
sp500_prices_df = stock_data.copy()
#

In [5]:
# Scrape large index funds using selium and eventually add to the df
url_categories = ["index-etfs", "sector-etfs", "commodity-etfs", "country-etfs"]
etfs = pd.DataFrame()

for url in url_categories:
    data = []
    driver = webdriver.Chrome()  
    driver.get(f"https://www.cnbc.com/{url}/")
    time.sleep(5)  
    
    # Locate the table
    table = driver.find_element(By.CLASS_NAME, "BasicTable-tableBody") # Class of table
    
    # Extract rows
    rows = table.find_elements(By.TAG_NAME, "tr")
    

    for row in rows:
        cols = [col.text for col in row.find_elements(By.TAG_NAME, "td")]
        if cols:
            data.append(cols)
    
    df = pd.DataFrame(data)
    df = df[df.columns[:2]]
    df.columns = ["Ticker", "Name"]
    df["Type"] = url
    display(df.head())
    print()


    etfs = pd.concat([etfs, df])
    
    
    driver.quit()

,Ticker,Name,Type
0,SPY,SPDR S&P 500 ETF Trust,index-etfs
1,QQQ,Invesco QQQ Trust,index-etfs
2,IWM,iShares Russell 2000 ETF,index-etfs
3,DIA,SPDR Dow Jones Industrial Average ETF Trust,index-etfs
4,VTI,Vanguard Total Stock Market Index Fund ETF Shares,index-etfs


,Ticker,Name,Type
0,XLE,Energy Select Sector SPDR Fund,sector-etfs
1,XLF,Financial Select SPDR Fund,sector-etfs
2,XLU,Utilities Select Sector SPDR Fund,sector-etfs
3,XLI,Industrial Select Sector SPDR Fund,sector-etfs
4,GDX,VanEck Gold Miners ETF,sector-etfs


,Ticker,Name,Type
0,GLD,SPDR Gold Shares,commodity-etfs
1,SLV,iShares Silver Trust,commodity-etfs
2,UNG,United States Natural Gas Fund,commodity-etfs
3,USO,"United States Oil Fund, LP",commodity-etfs
4,IAU,iShares Gold Trust,commodity-etfs


,Ticker,Name,Type
0,EWJ,iShares MSCI Japan ETF,country-etfs
1,FXI,iShares China Large-Cap ETF,country-etfs
2,EWZ,iShares MSCI Brazil ETF,country-etfs
3,EWT,iShares MSCI Taiwan ETF,country-etfs
4,DXJ,WisdomTree Japan Hedged Equity Fund,country-etfs


In [6]:
# Save important info
etfs_symbols = etfs.Ticker.tolist()
etf_type_dict = dict(zip(etfs["Ticker"], etfs["Type"]))
etfs.head()

,Ticker,Name,Type
0,SPY,SPDR S&P 500 ETF Trust,index-etfs
1,QQQ,Invesco QQQ Trust,index-etfs
2,IWM,iShares Russell 2000 ETF,index-etfs
3,DIA,SPDR Dow Jones Industrial Average ETF Trust,index-etfs
4,VTI,Vanguard Total Stock Market Index Fund ETF Shares,index-etfs


In [ ]:
# Get all etf data
etf_temp = pd.DataFrame()

# Same thing from above
start = time.time()
print("Loading...")
for e in etfs_symbols:
    temp_tick = Ticker(e)
    temp_df = temp_tick.yahoo_api_price(range='1y', dataGranularity='1d')
    temp_df = temp_df[["timestamp", "low", "high", "open", "close", "instrumentType"]]
    temp_df["Symbol"] = e

    etf_temp = pd.concat([etf_temp, temp_df])
end = time.time()

print(f"Finished: {end-start}")

Loading...


In [ ]:
#copy over
etf_prices = etf_temp.copy()
etf_prices = etf_prices.query('Symbol not in ["SIVR", "GDX", "SGOL"]')

In [ ]:
print("------------------------------------------- SP500 DATA -------------------------------------------")
display(sp500_prices_df.head(10))
print(); print()

print("-------------------------------------------- ETF Data --------------------------------------------")
display(etf_prices.head(10))

In [ ]:
# Concatonate both dfs
all_security_data = pd.concat([sp500_prices_df, etf_prices])
all_security_data = all_security_data.set_index("timestamp")

In [ ]:
# Get monthly_pct_change
all_security_data['(t+1)'] = all_security_data.groupby('Symbol')['close'].shift(-21) 
all_security_data["Monthly_Pct_Change"] = (all_security_data['(t+1)'] - all_security_data['close']) / all_security_data['close']
all_security_data = all_security_data.drop(columns = "(t+1)")

all_security_data

In [ ]:
# Plot well performing stocks
top_performers = all_security_data.groupby('Symbol', as_index = False)["Monthly_Pct_Change"].median()
top_performers = top_performers.sort_values(by = "Monthly_Pct_Change", ascending = False).head(5)

# Plot best performing stocks(based off of 1 month return) over the past 6 months
fig, ax = plt.subplots(5, 2, figsize = (18, 30))
#fig.suptitle("Best Performing Stocks Based on 1-Month Returns")
for i, symb in enumerate(top_performers.Symbol):
    
    temp = all_security_data.query('Symbol == @symb')
    temp["Monthly_Pct_Change"] = temp["Monthly_Pct_Change"] * 100
    temp.close.plot(ax = ax[i, 0]); ax[i, 0].set_title(f"{symb} Close Price")
    temp.Monthly_Pct_Change.plot(ax=ax[i, 1], linestyle = '--'); ax[i, 1].axhline(y = 0, color = 'k', linestyle = '--') 

    lab = f"Median Return: {round(float(top_performers.query('Symbol == @symb')['Monthly_Pct_Change']) * 100)}"
    ax[i, 1].axhline(y=float(top_performers.query('Symbol == @symb')['Monthly_Pct_Change'])*100, color='blue', linestyle='--', label=lab)
    ax[i, 1].legend()

    
    ax[i, 1].set_title(f"Monthly Returns on Investment for {symb}")
    
    ax[i,0].grid(linestyle = '--', alpha = 0.5, color = 'k'); ax[i,1].grid(linestyle = '--', alpha = 0.5, color = 'k')

plt.subplots_adjust(hspace = 0.5)

## Here, we will group stocks into buckets/graphs to diversify our portfolio. Each group theorectically will represent a sector or branch of the economy. We will do this grouping by seeing which stocks have the most similar monthly returns in terms of correlation

### We will use the networx library to make easy groupings based off of correlations of stock returns from month-to-month

In [ ]:
# Widen df to calculate correlations between monthly returns of stocks
stock_data_wide = all_security_data.dropna().pivot_table(
    index="timestamp",
    columns="Symbol",
    values="Monthly_Pct_Change"
)
stock_corr = stock_data_wide.corr().rename_axis(index="Ticker2", columns="Ticker1")

display(stock_corr)

In [ ]:
# Unstack to show direct comparisons
stock_corr = stock_corr.unstack().sort_values(ascending = False)
stock_corr = stock_corr[stock_corr < 0.999999]

In [ ]:
stock_corr_compare = stock_corr.copy()
stock_corr_compare = stock_corr_compare.reset_index()

stock_corr_compare.columns = ["Ticker1", "Ticker2", "Monthly_Pct_Change_Corr"]
stock_corr_compare["Pairs"] = stock_corr_compare.apply(
    lambda row: tuple(sorted([row["Ticker1"], row["Ticker2"]])), axis=1
)

stock_corr_compare = stock_corr_compare.drop_duplicates(subset = ["Pairs"])
stock_corr_compare = stock_corr_compare.drop(columns = "Pairs")

In [ ]:
stock_corr_compare.head(25)

In [ ]:
# -- Create Groups --
# Threshold for correlation
threshold = 0.8825 # 88.25% correlation

# Filter edges
df_filtered = stock_corr_compare[stock_corr_compare["Monthly_Pct_Change_Corr"] >= threshold]

# Build graph
G = nx.Graph()
for _, row in df_filtered.iterrows():
    G.add_edge(row["Ticker1"], row["Ticker2"], weight=row["Monthly_Pct_Change_Corr"])

# Connected components (looser groups)
clusters = list(nx.connected_components(G))
clusters = [list(c) for c in clusters]

print("Connected Component Clusters:")
for i, cluster in enumerate(clusters, 1):
    print(f"Cluster {i}: {cluster}")

In [ ]:
components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
n = len(components)
cols = 3  # number of columns in grid
rows = -(-n // cols)  # ceiling division

fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4))

# If only one subplot (e.g., 1 component), wrap axes in list
if n == 1:
    axes = [[axes]]
elif rows == 1:
    axes = [axes]  # make it 2D for consistency

axes = [ax for row_axes in axes for ax in (row_axes if isinstance(row_axes, (list, np.ndarray)) else [row_axes])]

for i, component in enumerate(components):
    ax = axes[i]
    pos = nx.spring_layout(component, seed=42)
    
    # Draw nodes, edges, and labels
    nx.draw(component, pos, ax=ax, with_labels=True, node_color='skyblue', node_size=700,
            edge_color='gray', width=[component[u][v]['weight'] * 5 for u, v in component.edges()])
    
    ax.set_title(f"Component {i+1}")
    ax.axis('off')

# Turn off unused axes
for j in range(len(components), len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# RISK FREE RETURN BASED OF OFF US TREASSURY NOTE
# ANNUALIZED RETURN FOR NOTE IS 4.33%
RISK_FREE_RETURN =  (4.33/100)/12 # Divide for 12 months in a year

def calc_sharpe_ratio(stocks, all_security_data):
    """
    Create a function to calculate the Sharpe ratios of a list of stocks. 
    Save top security and more if thresholds are met.

    Input: list of stocks
    Output: df
    """
    sharpe_ratios = []
    
    for stock in stocks:
        temp = all_security_data.query('Symbol == @stock')
        
        pct_change_avg = temp.Monthly_Pct_Change.mean()
        dev_returns = temp.Monthly_Pct_Change.std()
        
        if dev_returns == 0:
            print(f"Standard deviation is zero for {stock}, skipping...")
            continue

        monthly_sharpe_ratio = (pct_change_avg - RISK_FREE_RETURN) / dev_returns
        
        print(f"Sharpe Ratio (Monthly) for {stock}: {monthly_sharpe_ratio:.4f}")
        print(f"Sharpe Ratio (Annualized): {monthly_sharpe_ratio * np.sqrt(12):.4f}")
        print("-----------------------------------------------------------------------------------------")

        if monthly_sharpe_ratio >= 0.25:
            sharpe_ratios.append(
                (stock, monthly_sharpe_ratio, monthly_sharpe_ratio * np.sqrt(12))
            )
        print()
    
    if not sharpe_ratios:
        print("No good stocks")
        return

    sharpe_df = pd.DataFrame(sharpe_ratios, columns=["Symbol", "Monthly_Sharpe_Ratio", "Annualized_Sharpe_Ratio"])
    sharpe_df = sharpe_df.sort_values(by="Annualized_Sharpe_Ratio", ascending=False)

    top_sharpe_df = sharpe_df.head(1)
    filtered_remainder = sharpe_df[1:].query('Annualized_Sharpe_Ratio >= .9')

    if filtered_remainder.empty:
        return top_sharpe_df

    return pd.concat([top_sharpe_df, filtered_remainder])

# Initialize Portfolio df here with previously seen best performing stocks
portfolio_df = calc_sharpe_ratio(top_performers.Symbol, all_security_data)

In [ ]:
# Create Portfolio
for i, cluster in enumerate(clusters, 1):
    print(f"Cluster {i}")
    temp = calc_sharpe_ratio(cluster, all_security_data)
    display(temp)

    portfolio_df = pd.concat([portfolio_df, temp])
    
    print()

In [ ]:
# Create budget
BUDGET = 10000

In [ ]:
# Get close price
portfolio_df["Close"] = portfolio_df["Symbol"].apply(
    lambda symbol: float(all_security_data.query('Symbol == @symbol')['close'].iloc[-1])
)
# Get type of investment
portfolio_df["instrumentType"] = portfolio_df["Symbol"].apply(
    lambda symbol: str(all_security_data.query('Symbol == @symbol')['instrumentType'].iloc[0])
)
# Frac 
portfolio_df['frac_inv'] = portfolio_df["Monthly_Sharpe_Ratio"] / portfolio_df["Monthly_Sharpe_Ratio"].sum()
#number of shares
portfolio_df["n_shares"] = (BUDGET * portfolio_df['frac_inv']/portfolio_df['Close'])
#total amount to invest
portfolio_df["invt_amount"] = portfolio_df["n_shares"] * portfolio_df['Close']

portfolio_df = portfolio_df.sort_values(by = 'Annualized_Sharpe_Ratio', ascending = False) # Sort
portfolio_df = portfolio_df.drop_duplicates(subset = "Symbol") # Drop some that were added twice(top 5 performers)

In [ ]:
portfolio_df

In [ ]:
# Plot
fig, ax = plt.subplots(3, 1, figsize = (22, 30))

# Plot one
for sec in portfolio_df.Symbol:
    temp_df = all_security_data.query('Symbol == @sec')
    #temp_df.plot(x = temp.index, y = 'close', label = sec, ax=ax[0])
    ax[0].plot(temp_df.index, (temp_df['Monthly_Pct_Change'])*100, label = sec)
    
ax[0].legend()
ax[0].grid(linestyle = '--', alpha = 0.5, color = 'k')
ax[0].set_title("Monthly Returns of Securities")
ax[0].axhline(y = 0, color = 'k', linewidth = 4, linestyle = '--')

# Second Plot Etfs
sns.countplot(data=portfolio_df, x="instrumentType",ax=ax[1])
ax[1].set_xlabel("Instrument Type"); ax[1].set_ylabel("Count"); ax[1].set_title("Types of Investments")
ax[1].grid(linestyle = '--', alpha = 0.5, color = 'k')


# Third Plot
explode = [0.05] * len(portfolio_df)  # Slightly separate all slices

ax[2].pie(
    portfolio_df["invt_amount"],
    labels=portfolio_df["Symbol"],
    autopct='%1.1f%%',
    explode=explode,
    shadow=True,
    startangle=90,
    textprops={'fontsize': 10}
)

ax[2].set_title("Investment Allocation by Stock", fontsize=14)
ax[2].axis('equal')  # Make it a circle
plt.show()

# Thank you for viewing this notebook. In 1 month, I will create an update to see how our investments did

In [ ]:
# Save df 
portfolio_df.to_csv("sept-04-portfolio.csv", index = False)